In [9]:
import datetime
from dotenv import load_dotenv
load_dotenv('semantic-kernel/python/.env')
import os
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
model = "gpt-4-1106-preview"

# Wtih langchain

In [3]:
from langchain.agents import tool
@tool
def current_date()-> str:
    """Get today's date"""
    return datetime.datetime.now().strftime("%Y-%m-%d")

# mock API call
@tool
def get_order_status(order_id:str) -> str:
    """Get order delivery status and date"""
    if order_id == '1':
        order = {'item_name':'A', 'order_date':'2023-10-10', 'delivery_status':'completed', 'delivery_date':'2023-10-11'}
    if order_id == '2':
        order = {'item_name':'B', 'order_date':'2023-11-22', 'delivery_status':'completed', 'delivery_date':'2023-11-23'}
    if order_id == '3':
        order = {'item_name':'C', 'order_date':'2023-12-01', 'delivery_status':'in delivery', 'delivery_date':'2023-12-15'}
    return str(order)

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.schema.messages import AIMessage, HumanMessage

# component of agent
llm = ChatOpenAI(model=model, temperature=0)
tools = [current_date, get_order_status]
MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """You are chatbot who answers questions from EC customer about their orders and returns. 
        Return policy : accept returns within 30 days from the delivery date with full refund without asking reason."""),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

#llm + tool
llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

#Define agent
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

In [18]:
# Execute agent
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [19]:
# start a thread
chat_history = []
user_input = "What is the status of my order for B? order_id is 2."
result = agent_executor.invoke({"input": user_input, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=user_input),
        AIMessage(content=result["output"]),
    ]
)



> Entering new AgentExecutor chain...

Invoking: `get_order_status` with `{'order_id': '2'}`


{'item_name': 'B', 'order_date': '2023-11-22', 'delivery_status': 'completed', 'delivery_date': '2023-11-23'}The status of your order for item B with order ID 2 is completed. It was delivered on November 23, 2023. If you have any further questions or need assistance with your order, feel free to ask.

> Finished chain.


In [20]:
user_input = "Can I return this item?"
result = agent_executor.invoke({"input": user_input, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=user_input),
        AIMessage(content=result["output"]),
    ]
)



> Entering new AgentExecutor chain...

Invoking: `current_date` with `{}`


2023-11-30Yes, you can return the item. Since it was delivered on November 23, 2023, and today is November 30, 2023, you are within the 30-day return window from the delivery date. You can proceed with the return for a full refund without needing to provide a reason. If you need instructions on how to initiate the return process, please let me know, and I'll be happy to assist you further.

> Finished chain.
